# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]
print(len(text))

305203


## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 20)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249

The sentences 0 to 20:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.


Moe_Szyslak: Ah, isn't that nice. Now, there is a politician

## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    
    vocab = set(text)
    vocab_to_int = {c: i for i, c in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    print(int_to_vocab)
    print(vocab_to_int)
    # TODO: Implement Function
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

{0: "moe's", 1: 'yeah', 2: 'gonna', 3: 'mike', 4: 'your', 5: 'drink', 6: 'there', 7: 'has', 8: 'effervescent', 9: 'you', 10: 'an', 11: 'bart_simpson', 12: 'where', 13: 'i', 14: 'and', 15: 'problems', 16: 'skills', 17: 'the', 18: 'hello', 19: 'moe', 20: 'little', 21: 'forget', 22: 'homer', 23: 'should', 24: 'back', 25: 'got', 26: 'moe_szyslak', 27: 'self', 28: 'matter', 29: 'days', 30: 'one', 31: 'elite', 32: 'not', 33: 'is', 34: 'on', 35: 'barney_gumble', 36: 'check', 37: 'hey', 38: 'to', 39: 'meet', 40: 'pick', 41: 'of', 42: "i'll", 43: 'normal', 44: 'enhance', 45: 'eh', 46: 'seen', 47: 'these', 48: 'give', 49: 'only', 50: 'catch', 51: 'listen', 52: 'rotch', 53: 'carve', 54: 'ice', 55: 'anybody', 56: 'with', 57: 'last', 58: "i'm", 59: 'social', 60: 'tavern', 61: 'me', 62: 'whats', 63: 'my', 64: 'homer_simpson', 65: 'name', 66: 'hold', 67: 'lately', 68: "you're", 69: 'puke', 70: 'another'}
{"moe's": 0, 'yeah': 1, 'mike': 3, 'gonna': 2, 'has': 7, 'effervescent': 8, 'you': 9, "i'm": 58, 

### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    #text = text.lower()
    diction = {".": "||PERIOD||",",": "||COMMA||",'"': "||QUOTATION_MARK||",";": "||SEMICOLON||","!": "||EXCLAMATION_MARK||",
            "?": "||QUESTION_MARK||","(": "||LEFT_PAREN||",")": "||RIGHT_PAREN||","--": "||DASH||",
            "\n": "||NEW_LINE||"}

    # TODO: Implement Function
    print(diction)
    return diction
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

{';': '||SEMICOLON||', ',': '||COMMA||', '.': '||PERIOD||', '!': '||EXCLAMATION_MARK||', ')': '||RIGHT_PAREN||', '?': '||QUESTION_MARK||', '(': '||LEFT_PAREN||', '"': '||QUOTATION_MARK||', '--': '||DASH||', '\n': '||NEW_LINE||'}
Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

{';': '||SEMICOLON||', ',': '||COMMA||', '.': '||PERIOD||', '!': '||EXCLAMATION_MARK||', ')': '||RIGHT_PAREN||', '?': '||QUESTION_MARK||', '(': '||LEFT_PAREN||', '"': '||QUOTATION_MARK||', '--': '||DASH||', '\n': '||NEW_LINE||'}
{0: 'sweetly', 1: 'suave', 2: 'compliment', 3: 'escort', 4: 'add', 5: 'easter', 6: 'her', 7: 'inherent', 8: 'dreamy', 9: '_burns_heads:', 10: 'full-blooded', 11: 'by', 12: 'wolfcastle', 13: 'shaker', 14: 'dingy', 15: 'east', 16: 'playoff', 17: 'bitterly', 18: 'officer', 19: 'kearney_zzyzwicz:', 20: 'wrong', 21: 'mad', 22: 'looking', 23: 'anxious', 24: 'wreck', 25: 'drank', 26: "'bout", 27: 'teenage_barney:', 28: 'unlucky', 29: 'another', 30: 'dealer', 31: 'courthouse', 32: 'ninety-nine', 33: 'writers', 34: 'sedaris', 35: 'forty-nine', 36: "can't-believe-how-bald-he-is", 37: 'admirer', 38: 'forgive', 39: 'joining', 40: 'runs', 41: 'tons', 42: 'knees', 43: 'kind', 44: 'severe', 45: 'benjamin:', 46: 'onions', 47: 'buttons', 48: 'babar', 49: 'kucinich', 50: 'factor

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [40]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0


C:\Users\MKenneth\Miniconda3\envs\tensorfloww\lib\site-packages\ipykernel_launcher.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.
  


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [41]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    inputs = tf.placeholder(tf.int32,shape=[None,None], name='input')
    targets = tf.placeholder(tf.int32,shape=[None,None], name='targets')
    learningrate = tf.placeholder(tf.float32, name='learningrate')
    # TODO: Implement Function
    return inputs, targets, learningrate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [42]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    # Stack up multiple LSTM layers, for deep learning
    #drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=0.5)
    cell = tf.contrib.rnn.MultiRNNCell([lstm]*2)
    # Getting an initial state of all zeros
    
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state,name="initial_state")
    # TODO: Implement Function
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [50]:
def get_embed(input_data, vocab_size, embed_dim):
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    # TODO: Implement Function
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [51]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    #tensor, initial = get_init_cell(cell,inputs)
    outputs, state = tf.nn.dynamic_rnn(cell, inputs,dtype=tf.float32)
    final_state = tf.identity(state,name = "final_state")
    # TODO: Implement Function
    return outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [64]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    input_data = get_embed(input_data, vocab_size, embed_dim)
    outputs, FinalState = build_rnn(cell, input_data)
    Logits = tf.contrib.layers.fully_connected(outputs,vocab_size,activation_fn=None, weights_initializer=tf.truncated_normal_initializer(stddev=1.0))
    # TODO: Implement Function
    return Logits, FinalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [53]:
def get_batches(int_text, batch_size, seq_length):
#batch_size = 3
#seq_length = 2
#int_text = list(range(1000*seq_length))
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    num_batches = (len(int_text)//batch_size//seq_length)
    #print(int_text)
    batches = np.zeros((num_batches,2,batch_size,seq_length))
    #print(num_batches)
    #print(batch_size)
    #print(seq_length)
    for f in range(0,num_batches):
        for x in range(0,batch_size):
            for y in range(0,seq_length):
                g=y+x*num_batches*seq_length
                batches[f][0][x][y] = int_text[g+f*seq_length]
                batches[f][1][x][y] = int_text[g+1+f*seq_length]
                if (f==num_batches-1 and x==batch_size-1 and y==seq_length-1):
                    batches[f][1][x][y] = int_text[0]
            
    
    
    return batches



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [71]:
# Number of Epochs
num_epochs = 60
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 1024
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 20
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 1

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [72]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)
    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [73]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/26   train_loss = 9.101
Epoch   0 Batch    1/26   train_loss = 11.052
Epoch   0 Batch    2/26   train_loss = 9.057
Epoch   0 Batch    3/26   train_loss = 9.078
Epoch   0 Batch    4/26   train_loss = 8.411
Epoch   0 Batch    5/26   train_loss = 8.272
Epoch   0 Batch    6/26   train_loss = 8.045
Epoch   0 Batch    7/26   train_loss = 7.715
Epoch   0 Batch    8/26   train_loss = 7.775
Epoch   0 Batch    9/26   train_loss = 7.658
Epoch   0 Batch   10/26   train_loss = 7.366
Epoch   0 Batch   11/26   train_loss = 7.071
Epoch   0 Batch   12/26   train_loss = 7.099
Epoch   0 Batch   13/26   train_loss = 6.979
Epoch   0 Batch   14/26   train_loss = 6.824
Epoch   0 Batch   15/26   train_loss = 6.703
Epoch   0 Batch   16/26   train_loss = 6.812
Epoch   0 Batch   17/26   train_loss = 6.680
Epoch   0 Batch   18/26   train_loss = 6.639
Epoch   0 Batch   19/26   train_loss = 6.532
Epoch   0 Batch   20/26   train_loss = 6.483
Epoch   0 Batch   21/26   train_loss = 6.339
Epoch   0

Epoch   7 Batch    1/26   train_loss = 3.140
Epoch   7 Batch    2/26   train_loss = 2.935
Epoch   7 Batch    3/26   train_loss = 3.030
Epoch   7 Batch    4/26   train_loss = 2.949
Epoch   7 Batch    5/26   train_loss = 2.965
Epoch   7 Batch    6/26   train_loss = 2.977
Epoch   7 Batch    7/26   train_loss = 2.943
Epoch   7 Batch    8/26   train_loss = 2.928
Epoch   7 Batch    9/26   train_loss = 3.028
Epoch   7 Batch   10/26   train_loss = 2.834
Epoch   7 Batch   11/26   train_loss = 2.836
Epoch   7 Batch   12/26   train_loss = 2.849
Epoch   7 Batch   13/26   train_loss = 2.853
Epoch   7 Batch   14/26   train_loss = 2.770
Epoch   7 Batch   15/26   train_loss = 2.671
Epoch   7 Batch   16/26   train_loss = 2.692
Epoch   7 Batch   17/26   train_loss = 2.782
Epoch   7 Batch   18/26   train_loss = 2.857
Epoch   7 Batch   19/26   train_loss = 2.731
Epoch   7 Batch   20/26   train_loss = 2.713
Epoch   7 Batch   21/26   train_loss = 2.759
Epoch   7 Batch   22/26   train_loss = 2.676
Epoch   7 

Epoch  14 Batch    2/26   train_loss = 0.816
Epoch  14 Batch    3/26   train_loss = 0.811
Epoch  14 Batch    4/26   train_loss = 0.773
Epoch  14 Batch    5/26   train_loss = 0.787
Epoch  14 Batch    6/26   train_loss = 0.753
Epoch  14 Batch    7/26   train_loss = 0.784
Epoch  14 Batch    8/26   train_loss = 0.770
Epoch  14 Batch    9/26   train_loss = 0.755
Epoch  14 Batch   10/26   train_loss = 0.724
Epoch  14 Batch   11/26   train_loss = 0.791
Epoch  14 Batch   12/26   train_loss = 0.776
Epoch  14 Batch   13/26   train_loss = 0.766
Epoch  14 Batch   14/26   train_loss = 0.745
Epoch  14 Batch   15/26   train_loss = 0.679
Epoch  14 Batch   16/26   train_loss = 0.729
Epoch  14 Batch   17/26   train_loss = 0.733
Epoch  14 Batch   18/26   train_loss = 0.742
Epoch  14 Batch   19/26   train_loss = 0.704
Epoch  14 Batch   20/26   train_loss = 0.722
Epoch  14 Batch   21/26   train_loss = 0.769
Epoch  14 Batch   22/26   train_loss = 0.728
Epoch  14 Batch   23/26   train_loss = 0.695
Epoch  14 

Epoch  21 Batch    3/26   train_loss = 0.296
Epoch  21 Batch    4/26   train_loss = 0.285
Epoch  21 Batch    5/26   train_loss = 0.258
Epoch  21 Batch    6/26   train_loss = 0.279
Epoch  21 Batch    7/26   train_loss = 0.307
Epoch  21 Batch    8/26   train_loss = 0.288
Epoch  21 Batch    9/26   train_loss = 0.293
Epoch  21 Batch   10/26   train_loss = 0.274
Epoch  21 Batch   11/26   train_loss = 0.265
Epoch  21 Batch   12/26   train_loss = 0.290
Epoch  21 Batch   13/26   train_loss = 0.244
Epoch  21 Batch   14/26   train_loss = 0.285
Epoch  21 Batch   15/26   train_loss = 0.312
Epoch  21 Batch   16/26   train_loss = 0.291
Epoch  21 Batch   17/26   train_loss = 0.262
Epoch  21 Batch   18/26   train_loss = 0.257
Epoch  21 Batch   19/26   train_loss = 0.270
Epoch  21 Batch   20/26   train_loss = 0.273
Epoch  21 Batch   21/26   train_loss = 0.275
Epoch  21 Batch   22/26   train_loss = 0.314
Epoch  21 Batch   23/26   train_loss = 0.241
Epoch  21 Batch   24/26   train_loss = 0.271
Epoch  21 

Epoch  28 Batch    4/26   train_loss = 0.212
Epoch  28 Batch    5/26   train_loss = 0.182
Epoch  28 Batch    6/26   train_loss = 0.192
Epoch  28 Batch    7/26   train_loss = 0.203
Epoch  28 Batch    8/26   train_loss = 0.185
Epoch  28 Batch    9/26   train_loss = 0.194
Epoch  28 Batch   10/26   train_loss = 0.180
Epoch  28 Batch   11/26   train_loss = 0.207
Epoch  28 Batch   12/26   train_loss = 0.199
Epoch  28 Batch   13/26   train_loss = 0.174
Epoch  28 Batch   14/26   train_loss = 0.205
Epoch  28 Batch   15/26   train_loss = 0.212
Epoch  28 Batch   16/26   train_loss = 0.209
Epoch  28 Batch   17/26   train_loss = 0.202
Epoch  28 Batch   18/26   train_loss = 0.208
Epoch  28 Batch   19/26   train_loss = 0.189
Epoch  28 Batch   20/26   train_loss = 0.219
Epoch  28 Batch   21/26   train_loss = 0.204
Epoch  28 Batch   22/26   train_loss = 0.186
Epoch  28 Batch   23/26   train_loss = 0.183
Epoch  28 Batch   24/26   train_loss = 0.183
Epoch  28 Batch   25/26   train_loss = 0.192
Epoch  29 

Epoch  35 Batch    5/26   train_loss = 0.170
Epoch  35 Batch    6/26   train_loss = 0.181
Epoch  35 Batch    7/26   train_loss = 0.187
Epoch  35 Batch    8/26   train_loss = 0.166
Epoch  35 Batch    9/26   train_loss = 0.185
Epoch  35 Batch   10/26   train_loss = 0.173
Epoch  35 Batch   11/26   train_loss = 0.178
Epoch  35 Batch   12/26   train_loss = 0.185
Epoch  35 Batch   13/26   train_loss = 0.148
Epoch  35 Batch   14/26   train_loss = 0.193
Epoch  35 Batch   15/26   train_loss = 0.198
Epoch  35 Batch   16/26   train_loss = 0.194
Epoch  35 Batch   17/26   train_loss = 0.182
Epoch  35 Batch   18/26   train_loss = 0.185
Epoch  35 Batch   19/26   train_loss = 0.189
Epoch  35 Batch   20/26   train_loss = 0.183
Epoch  35 Batch   21/26   train_loss = 0.190
Epoch  35 Batch   22/26   train_loss = 0.176
Epoch  35 Batch   23/26   train_loss = 0.171
Epoch  35 Batch   24/26   train_loss = 0.180
Epoch  35 Batch   25/26   train_loss = 0.172
Epoch  36 Batch    0/26   train_loss = 0.185
Epoch  36 

Epoch  42 Batch    6/26   train_loss = 0.174
Epoch  42 Batch    7/26   train_loss = 0.187
Epoch  42 Batch    8/26   train_loss = 0.174
Epoch  42 Batch    9/26   train_loss = 0.216
Epoch  42 Batch   10/26   train_loss = 0.168
Epoch  42 Batch   11/26   train_loss = 0.180
Epoch  42 Batch   12/26   train_loss = 0.193
Epoch  42 Batch   13/26   train_loss = 0.156
Epoch  42 Batch   14/26   train_loss = 0.185
Epoch  42 Batch   15/26   train_loss = 0.186
Epoch  42 Batch   16/26   train_loss = 0.190
Epoch  42 Batch   17/26   train_loss = 0.196
Epoch  42 Batch   18/26   train_loss = 0.182
Epoch  42 Batch   19/26   train_loss = 0.180
Epoch  42 Batch   20/26   train_loss = 0.180
Epoch  42 Batch   21/26   train_loss = 0.185
Epoch  42 Batch   22/26   train_loss = 0.188
Epoch  42 Batch   23/26   train_loss = 0.174
Epoch  42 Batch   24/26   train_loss = 0.171
Epoch  42 Batch   25/26   train_loss = 0.189
Epoch  43 Batch    0/26   train_loss = 0.193
Epoch  43 Batch    1/26   train_loss = 0.186
Epoch  43 

Epoch  49 Batch    7/26   train_loss = 0.340
Epoch  49 Batch    8/26   train_loss = 0.297
Epoch  49 Batch    9/26   train_loss = 0.381
Epoch  49 Batch   10/26   train_loss = 0.383
Epoch  49 Batch   11/26   train_loss = 0.349
Epoch  49 Batch   12/26   train_loss = 0.374
Epoch  49 Batch   13/26   train_loss = 0.331
Epoch  49 Batch   14/26   train_loss = 0.373
Epoch  49 Batch   15/26   train_loss = 0.386
Epoch  49 Batch   16/26   train_loss = 0.393
Epoch  49 Batch   17/26   train_loss = 0.353
Epoch  49 Batch   18/26   train_loss = 0.384
Epoch  49 Batch   19/26   train_loss = 0.427
Epoch  49 Batch   20/26   train_loss = 0.413
Epoch  49 Batch   21/26   train_loss = 0.363
Epoch  49 Batch   22/26   train_loss = 0.388
Epoch  49 Batch   23/26   train_loss = 0.376
Epoch  49 Batch   24/26   train_loss = 0.359
Epoch  49 Batch   25/26   train_loss = 0.383
Epoch  50 Batch    0/26   train_loss = 0.345
Epoch  50 Batch    1/26   train_loss = 0.355
Epoch  50 Batch    2/26   train_loss = 0.403
Epoch  50 

Epoch  56 Batch    8/26   train_loss = 0.172
Epoch  56 Batch    9/26   train_loss = 0.180
Epoch  56 Batch   10/26   train_loss = 0.167
Epoch  56 Batch   11/26   train_loss = 0.181
Epoch  56 Batch   12/26   train_loss = 0.182
Epoch  56 Batch   13/26   train_loss = 0.152
Epoch  56 Batch   14/26   train_loss = 0.186
Epoch  56 Batch   15/26   train_loss = 0.189
Epoch  56 Batch   16/26   train_loss = 0.196
Epoch  56 Batch   17/26   train_loss = 0.192
Epoch  56 Batch   18/26   train_loss = 0.184
Epoch  56 Batch   19/26   train_loss = 0.181
Epoch  56 Batch   20/26   train_loss = 0.174
Epoch  56 Batch   21/26   train_loss = 0.177
Epoch  56 Batch   22/26   train_loss = 0.172
Epoch  56 Batch   23/26   train_loss = 0.172
Epoch  56 Batch   24/26   train_loss = 0.183
Epoch  56 Batch   25/26   train_loss = 0.165
Epoch  57 Batch    0/26   train_loss = 0.178
Epoch  57 Batch    1/26   train_loss = 0.181
Epoch  57 Batch    2/26   train_loss = 0.186
Epoch  57 Batch    3/26   train_loss = 0.179
Epoch  57 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [74]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [117]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [118]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    with loaded_graph.as_default():
        InputTensor = loaded_graph.get_tensor_by_name("input:0")
        InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
        FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
        ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
        
    # TODO: Implement Function
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [155]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    predword = int(np.random.choice(len(probabilities),1,p=probabilities))
    predword = int_to_vocab[predword]
    return predword


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [156]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)
    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)
    
    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})
    
    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])
        
        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)
        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak:(delighted) ooh, a text.(checks phone) let's see... text message for the rest of freedom.
kent_brockman:(threatening) well, homer, i thought you said that.
moe_szyslak:(married) ah, good is.
ned_flanders: it is the" code" it's coat ninety-nine", what'll" harvey it."
moe_szyslak: no, you got the wrong guy.
homer_simpson:(awkwardly) you dropped somethin'.
moe_szyslak: c'mon, i'm sorry, uncle. experiments, i didn't close up my day...(homer) i should be mad at it, moe.(gun) whoa, whoa, whoa. you said no fuss.
health_inspector: here, i'm looking for my viva here... which means"...
moe_szyslak:... the last angry clown. / canyonero is this hourly with a wheel") hold the sensible fire, truck.


carny: big hole, when are we gonna do it, moe.
moe_szyslak:(low


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.